# Dummy

In [2]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# Lire les données
app_train = pd.read_csv('data/application_train_split.csv')
app_test = pd.read_csv('data/application_test_split.csv')

# Séparer les données d'entraînement
if 'TARGET' in app_train:
    train = app_train.drop(columns=['TARGET'])
    train_labels = app_train['TARGET']
else:
    raise ValueError("La colonne 'TARGET' n'existe pas dans les données d'entraînement.")

# Préparer les colonnes numériques
numeric_cols = train.select_dtypes(include=['number']).columns

# Imputer les valeurs manquantes pour les colonnes numériques
imputer = SimpleImputer(strategy='median')
train_numeric = imputer.fit_transform(train[numeric_cols])
test_numeric = imputer.transform(app_test[numeric_cols])

# Normaliser les colonnes numériques
scaler = MinMaxScaler(feature_range=(0, 1))
train_numeric = scaler.fit_transform(train_numeric)
test_numeric = scaler.transform(test_numeric)

# Créer et entraîner le DummyClassifier
dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(train_numeric, train_labels)

# Faire des prédictions sur les données d'entraînement (pour évaluation)
train_predictions = dummy_clf.predict(train_numeric)

# Calculer le score de précision sur les données d'entraînement
train_score = accuracy_score(train_labels, train_predictions)

# Faire des prédictions sur les données de test
dummy_predictions = dummy_clf.predict(test_numeric)

# Créer le DataFrame de soumission
submit = app_test[['SK_ID_CURR']].copy()
submit['TARGET'] = dummy_predictions

# Afficher les résultats
print("Score du modèle sur les données d'entraînement :", train_score)
print(submit.head())


Score du modèle sur les données d'entraînement : 0.8523671704056082
   SK_ID_CURR  TARGET
0      342180       0
1      259636       0
2      305882       0
3      243264       0
4      264946       0


# Regression logistique

In [513]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import pandas as pd

# Charger les données
app_train = pd.read_csv('data/application_train_split.csv')
app_test = pd.read_csv('data/application_test_split.csv')

# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(columns=['TARGET'])
    train_labels = app_train['TARGET']
else:
    train = app_train.copy()

# Séparer les colonnes numériques et non numériques
numeric_cols = train.select_dtypes(include=['number']).columns
categorical_cols = train.select_dtypes(exclude=['number']).columns

# Imputation des valeurs manquantes pour les colonnes numériques
imputer = SimpleImputer(strategy='median')
train_numeric = imputer.fit_transform(train[numeric_cols])
test_numeric = imputer.transform(app_test[numeric_cols])

# Normalisation des colonnes numériques
scaler = MinMaxScaler(feature_range=(0, 1))
train_numeric = scaler.fit_transform(train_numeric)
test_numeric = scaler.transform(test_numeric)
train_final = pd.DataFrame(train_numeric, columns=numeric_cols)
test_final = pd.DataFrame(test_numeric, columns=numeric_cols)

# Créer et entraîner le modèle
log_reg = LogisticRegression(C=0.0001)
log_reg.fit(train_final, train_labels)

# Faire des prédictions sur les données d'entraînement
train_pred = log_reg.predict_proba(train_final)[:, 1]

# Calculer le score ROC AUC sur les données d'entraînement
roc_auc = roc_auc_score(train_labels, train_pred)

# Afficher le score ROC AUC
print(f'ROC AUC Score: {roc_auc:.4f}')

# Faire des prédictions sur l'ensemble de test
log_reg_pred = log_reg.predict_proba(test_final)[:, 1]

# Créer le DataFrame de soumission
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()


ROC AUC Score: 0.6831


/var/folders/25/t6v1433n06x6vfl9wjbxn8sw0000gn/T/ipykernel_43323/2695516506.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['TARGET'] = log_reg_pred


,SK_ID_CURR,TARGET
0,342180,0.081685
1,259636,0.064739
2,305882,0.077060
3,243264,0.063606
4,264946,0.064830


# Arbre de décision

In [515]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import pandas as pd

# Charger les données
app_train = pd.read_csv('data/application_train_split.csv')
app_test = pd.read_csv('data/application_test_split.csv')

# Convertir toutes les colonnes en numériques en utilisant LabelEncoder
label_encoders = {}
for col in app_train.columns:
    if app_train[col].dtype == 'object':
        le = LabelEncoder()
        app_train[col] = le.fit_transform(app_train[col].astype(str))
        if col in app_test.columns:
            app_test[col] = le.transform(app_test[col].astype(str))
        label_encoders[col] = le

# Séparer les données d'entraînement et les labels
if 'TARGET' in app_train:
    train = app_train.drop(columns=['TARGET'])
    train_labels = app_train['TARGET']
else:
    train = app_train.copy()
    train_labels = None

# Préparer les données de test sans l'identifiant
test = app_test.drop(columns=['SK_ID_CURR'])

# S'assurer que les colonnes sont alignées avant l'imputation
common_cols = train.columns.intersection(test.columns)
train = train[common_cols]
test = test[common_cols]

# Imputer les valeurs manquantes avec la médiane
imputer = SimpleImputer(strategy='median')
train = imputer.fit_transform(train)
test = imputer.transform(test)

# Créer le modèle d'Arbre de Décision avec des hyperparamètres
decision_tree = DecisionTreeClassifier(max_depth=7, min_samples_split=10, min_samples_leaf=5)

# Entraîner le modèle
decision_tree.fit(train, train_labels)

# Faire des prédictions sur les données d'entraînement
train_pred = decision_tree.predict_proba(train)[:, 1]

# Calculer le score ROC AUC sur les données d'entraînement
roc_auc = roc_auc_score(train_labels, train_pred)

# Afficher le score ROC AUC
print(f'ROC AUC Score: {roc_auc:.4f}')

# Extraire l'importance des caractéristiques
feature_importance_values = decision_tree.feature_importances_
feature_importances = pd.DataFrame({'feature': common_cols, 'importance': feature_importance_values})

# Faire des prédictions sur l'ensemble de test
predictions = decision_tree.predict_proba(test)[:, 1]

# Créer le DataFrame de soumission
submit = pd.DataFrame({'SK_ID_CURR': app_test['SK_ID_CURR'], 'TARGET': predictions})

print("Prédictions:")
print(submit.head())


ROC AUC Score: 0.7277
Prédictions:
   SK_ID_CURR    TARGET
0      342180  0.045878
1      259636  0.079470
2      305882  0.045878
3      243264  0.037365
4      264946  0.061666


# Random Forest

In [517]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import pandas as pd

# Charger les données
app_train = pd.read_csv('data/application_train_split.csv')
app_test = pd.read_csv('data/application_test_split.csv')

# Convertir toutes les colonnes en numériques en utilisant LabelEncoder
label_encoders = {}
for col in app_train.columns:
    if app_train[col].dtype == 'object':
        le = LabelEncoder()
        app_train[col] = le.fit_transform(app_train[col].astype(str))
        if col in app_test.columns:
            app_test[col] = le.transform(app_test[col].astype(str))
        label_encoders[col] = le

# Séparer les données d'entraînement et les labels
if 'TARGET' in app_train:
    train = app_train.drop(columns=['TARGET'])
    train_labels = app_train['TARGET']
else:
    train = app_train.copy()
    train_labels = None

# Préparer les données de test sans l'identifiant
test = app_test.drop(columns=['SK_ID_CURR'])

# S'assurer que les colonnes sont alignées avant l'imputation
common_cols = train.columns.intersection(test.columns)
train = train[common_cols]
test = test[common_cols]

# Imputer les valeurs manquantes avec la médiane
imputer = SimpleImputer(strategy='median')
train = imputer.fit_transform(train)
test = imputer.transform(test)

# Créer le modèle RandomForest
random_forest = RandomForestClassifier(n_estimators=1, verbose=1, n_jobs=-1)

# Entraîner le modèle
random_forest.fit(train, train_labels)

# Faire des prédictions sur les données d'entraînement
train_pred = random_forest.predict_proba(train)[:, 1]

# Calculer le score ROC AUC sur les données d'entraînement
roc_auc = roc_auc_score(train_labels, train_pred)

# Afficher le score ROC AUC
print(f'ROC AUC Score: {roc_auc:.4f}')

# Extraire l'importance des caractéristiques
feature_importance_values = random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature': common_cols, 'importance': feature_importance_values})

# Faire des prédictions sur l'ensemble de test
predictions = random_forest.predict_proba(test)[:, 1]

# Créer le DataFrame de soumission
submit = pd.DataFrame({'SK_ID_CURR': app_test['SK_ID_CURR'], 'TARGET': predictions})

print("Prédictions:")
print(submit.head())

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.


ROC AUC Score: 0.8230
Prédictions:
   SK_ID_CURR  TARGET
0      342180     0.0
1      259636     0.0
2      305882     0.0
3      243264     0.0
4      264946     0.0


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


# Fonction unique

In [519]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
import lightgbm as lgb
import logging
import sys

def process_and_predict(train_file, test_file):
    # Configurer le logging pour ignorer les messages d'information de LightGBM
    logging.getLogger('lightgbm').setLevel(logging.ERROR)
    
    # Rediriger stdout pour éviter les messages d'information de LightGBM
    original_stdout = sys.stdout
    sys.stdout = open('/dev/null', 'w')  # Sur Unix/Linux/Mac
    # sys.stdout = open(os.devnull, 'w')  # Sur Windows
    
    # Charger les données
    app_train = pd.read_csv(train_file)
    app_test = pd.read_csv(test_file)
    
    # Convertir toutes les colonnes en numériques en utilisant LabelEncoder
    label_encoders = {}
    for col in app_train.columns:
        if app_train[col].dtype == 'object':
            le = LabelEncoder()
            app_train[col] = le.fit_transform(app_train[col].astype(str))
            if col in app_test.columns:
                app_test[col] = le.transform(app_test[col].astype(str))
            label_encoders[col] = le
    
    # Séparer les données d'entraînement et les labels
    if 'TARGET' in app_train:
        train = app_train.drop(columns=['TARGET'])
        train_labels = app_train['TARGET']
    else:
        raise ValueError("La colonne 'TARGET' n'existe pas dans les données d'entraînement.")
    
    # Préparer les données de test sans l'identifiant
    test = app_test.drop(columns=['SK_ID_CURR'])
    
    # S'assurer que les colonnes sont alignées avant l'imputation
    common_cols = train.columns.intersection(test.columns)
    train = train[common_cols]
    test = test[common_cols]
    
    # Imputer les valeurs manquantes avec la médiane
    imputer = SimpleImputer(strategy='median')
    train_numeric = imputer.fit_transform(train)
    test_numeric = imputer.transform(test)
    
    # Normaliser les colonnes numériques
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_numeric = scaler.fit_transform(train_numeric)
    test_numeric = scaler.transform(test_numeric)
    
    # Convertir les matrices numpy en DataFrames pandas
    train_final = pd.DataFrame(train_numeric, columns=common_cols)
    test_final = pd.DataFrame(test_numeric, columns=common_cols)
    
    # Créer et entraîner le DummyClassifier
    dummy_clf = DummyClassifier(strategy="stratified")
    dummy_clf.fit(train_final, train_labels)
    dummy_predictions = dummy_clf.predict(test_final)
    dummy_score = dummy_clf.score(train_final, train_labels)
    
    # Créer et entraîner le modèle LogisticRegression
    log_reg = LogisticRegression(C=0.0001, max_iter=1000)
    log_reg.fit(train_final, train_labels)
    train_pred_log_reg = log_reg.predict_proba(train_final)[:, 1]
    roc_auc_log_reg = roc_auc_score(train_labels, train_pred_log_reg)
    
    # Créer et entraîner le modèle DecisionTreeClassifier
    decision_tree = DecisionTreeClassifier(max_depth=7, min_samples_split=10, min_samples_leaf=5)
    decision_tree.fit(train_final, train_labels)
    train_pred_tree = decision_tree.predict_proba(train_final)[:, 1]
    roc_auc_tree = roc_auc_score(train_labels, train_pred_tree)
    
    # Créer et entraîner le modèle RandomForestClassifier
    random_forest = RandomForestClassifier(n_estimators=1, verbose=1, n_jobs=-1)
    random_forest.fit(train_final, train_labels)
    train_pred_rf = random_forest.predict_proba(train_final)[:, 1]
    roc_auc_rf = roc_auc_score(train_labels, train_pred_rf)
    
    # Créer et entraîner le modèle CatBoostClassifier
    catboost_model = CatBoostClassifier(iterations=100, depth=7, learning_rate=0.1, loss_function='Logloss', verbose=0)
    catboost_model.fit(train_final, train_labels)
    train_pred_catboost = catboost_model.predict_proba(train_final)[:, 1]
    roc_auc_catboost = roc_auc_score(train_labels, train_pred_catboost)
    catboost_predictions = catboost_model.predict_proba(test_final)[:, 1]
    
    # Créer et entraîner le modèle LightGBMClassifier
    lightgbm_model = lgb.LGBMClassifier(n_estimators=100, max_depth=7, learning_rate=0.1)
    lightgbm_model.fit(train_final, train_labels)
    train_pred_lightgbm = lightgbm_model.predict_proba(train_final)[:, 1]
    roc_auc_lightgbm = roc_auc_score(train_labels, train_pred_lightgbm)
    lightgbm_predictions = lightgbm_model.predict_proba(test_final)[:, 1]
    
    # Rétablir stdout
    sys.stdout = original_stdout
    
    # Afficher les résultats dans un tableau lisible
    results = {
        'Model': ['DummyClassifier', 'LogisticRegression', 'DecisionTreeClassifier', 'RandomForestClassifier', 'CatBoostClassifier', 'LightGBMClassifier'],
        'ROC AUC Score': [dummy_score, roc_auc_log_reg, roc_auc_tree, roc_auc_rf, roc_auc_catboost, roc_auc_lightgbm]
    }
    results_df = pd.DataFrame(results)
    
    print("Scores ROC AUC des modèles :")
    print(results_df.to_string(index=False))
    
    return {
        'DummyClassifier': dummy_predictions,
        'LogisticRegression': log_reg.predict_proba(test_final)[:, 1],
        'DecisionTreeClassifier': decision_tree.predict_proba(test_final)[:, 1],
        'RandomForestClassifier': random_forest.predict_proba(test_final)[:, 1],
        'CatBoostClassifier': catboost_predictions,
        'LightGBMClassifier': lightgbm_predictions
    }

# Appel de la fonction avec les chemins des fichiers
predictions = process_and_predict('data/application_train_split.csv', 'data/application_test_split.csv')


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Scores ROC AUC des modèles :
                 Model  ROC AUC Score
       DummyClassifier       0.852121
    LogisticRegression       0.685603
DecisionTreeClassifier       0.727685
RandomForestClassifier       0.829548
    CatBoostClassifier       0.768578
    LightGBMClassifier       0.798777


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


# Fonction unique avec CV

In [521]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
import lightgbm as lgb
import logging
import sys
import time
from tabulate import tabulate

def measure_prediction_time(model, X, n_predictions=1000):
    """Mesurer le temps de prédiction moyen sur n_predictions échantillons."""
    start_time = time.time()
    for _ in range(n_predictions):
        _ = model.predict_proba(X)[:, 1]
    end_time = time.time()
    elapsed_time = end_time - start_time
    return "{:.2f} s".format(elapsed_time)

def process_and_predict(train_file, test_file):
    # Configurer le logging pour ignorer les messages d'information de LightGBM
    logging.getLogger('lightgbm').setLevel(logging.ERROR)
    
    # Rediriger stdout pour éviter les messages d'information de LightGBM
    original_stdout = sys.stdout
    sys.stdout = open('/dev/null', 'w')  # Sur Unix/Linux/Mac
    
    # Charger les données
    app_train = pd.read_csv(train_file)
    app_test = pd.read_csv(test_file)
    
    # Convertir toutes les colonnes en numériques en utilisant LabelEncoder
    label_encoders = {}
    for col in app_train.columns:
        if app_train[col].dtype == 'object':
            le = LabelEncoder()
            app_train[col] = le.fit_transform(app_train[col].astype(str))
            if col in app_test.columns:
                app_test[col] = le.transform(app_test[col].astype(str))
            label_encoders[col] = le
    
    # Séparer les données d'entraînement et les labels
    if 'TARGET' in app_train:
        train = app_train.drop(columns=['TARGET'])
        train_labels = app_train['TARGET']
    else:
        raise ValueError("La colonne 'TARGET' n'existe pas dans les données d'entraînement.")
    
    # Préparer les données de test sans l'identifiant
    test = app_test.drop(columns=['SK_ID_CURR'])
    
    # S'assurer que les colonnes sont alignées avant l'imputation
    common_cols = train.columns.intersection(test.columns)
    train = train[common_cols]
    test = test[common_cols]
    
    # Imputer les valeurs manquantes avec la médiane
    imputer = SimpleImputer(strategy='median')
    train_numeric = imputer.fit_transform(train)
    test_numeric = imputer.transform(test)
    
    # Normaliser les colonnes numériques
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_numeric = scaler.fit_transform(train_numeric)
    test_numeric = scaler.transform(test_numeric)
    
    # Convertir les matrices numpy en DataFrames pandas
    train_final = pd.DataFrame(train_numeric, columns=common_cols)
    test_final = pd.DataFrame(test_numeric, columns=common_cols)
    
    # Créer et entraîner le DummyClassifier
    dummy_clf = DummyClassifier(strategy="stratified")
    dummy_clf.fit(train_final, train_labels)
    dummy_predictions = dummy_clf.predict_proba(test_final)[:, 1]
    dummy_auc = roc_auc_score(train_labels, dummy_clf.predict_proba(train_final)[:, 1])
    dummy_time = measure_prediction_time(dummy_clf, test_final)
    
    # Définir les paramètres pour GridSearchCV pour LogisticRegression
    log_reg_params = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10]}
    log_reg = LogisticRegression(max_iter=2000)  # Augmenté à 2000
    log_reg_grid = GridSearchCV(log_reg, log_reg_params, cv=5, scoring='roc_auc')
    log_reg_grid.fit(train_final, train_labels)
    log_reg_auc = roc_auc_score(train_labels, log_reg_grid.predict_proba(train_final)[:, 1])
    log_reg_time = measure_prediction_time(log_reg_grid, test_final)
    
    # Définir les paramètres pour GridSearchCV pour DecisionTreeClassifier
    tree_params = {'max_depth': [5, 7, 10], 'min_samples_split': [10, 20, 50]}
    decision_tree = DecisionTreeClassifier()
    tree_grid = GridSearchCV(decision_tree, tree_params, cv=5, scoring='roc_auc')
    tree_grid.fit(train_final, train_labels)
    tree_auc = roc_auc_score(train_labels, tree_grid.predict_proba(train_final)[:, 1])
    tree_time = measure_prediction_time(tree_grid, test_final)
    
    # Définir les paramètres pour GridSearchCV pour RandomForestClassifier
    rf_params = {'n_estimators': [10, 50, 100], 'max_depth': [5, 7, 10]}
    random_forest = RandomForestClassifier(n_jobs=-1)
    rf_grid = GridSearchCV(random_forest, rf_params, cv=5, scoring='roc_auc')
    rf_grid.fit(train_final, train_labels)
    rf_auc = roc_auc_score(train_labels, rf_grid.predict_proba(train_final)[:, 1])
    rf_time = measure_prediction_time(rf_grid, test_final)
    
    # Définir les paramètres pour GridSearchCV pour CatBoostClassifier
    catboost_params = {'depth': [1,2, 3,4, 5], 'learning_rate': [0.01, 0.1, 0.3]}
    catboost_model = CatBoostClassifier(iterations=100, loss_function='Logloss', verbose=0)
    catboost_grid = GridSearchCV(catboost_model, catboost_params, cv=5, scoring='roc_auc')
    catboost_grid.fit(train_final, train_labels)
    catboost_auc = roc_auc_score(train_labels, catboost_grid.predict_proba(train_final)[:, 1])
    catboost_predictions = catboost_grid.predict_proba(test_final)[:, 1]
    catboost_time = measure_prediction_time(catboost_grid, test_final)
    
    # Définir les paramètres pour GridSearchCV pour LightGBMClassifier
    lightgbm_params = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [1,2, 3,4, 5],
        'num_leaves': [2**6 - 1, 2**7 - 1, 2**8 - 1]  # Ajusté pour respecter la condition
    }
    lightgbm_model = lgb.LGBMClassifier()
    lightgbm_grid = GridSearchCV(lightgbm_model, lightgbm_params, cv=5, scoring='roc_auc')
    lightgbm_grid.fit(train_final, train_labels)
    lightgbm_auc = roc_auc_score(train_labels, lightgbm_grid.predict_proba(train_final)[:, 1])
    lightgbm_predictions = lightgbm_grid.predict_proba(test_final)[:, 1]
    lightgbm_time = measure_prediction_time(lightgbm_grid, test_final)
    
    # Rétablir stdout
    sys.stdout = original_stdout

    # Afficher les meilleurs hyperparamètres pour chaque modèle
    best_params = {
        'LogisticRegression': log_reg_grid.best_params_,
        'DecisionTreeClassifier': tree_grid.best_params_,
        'RandomForestClassifier': rf_grid.best_params_,
        'CatBoostClassifier': catboost_grid.best_params_,
        'LightGBMClassifier': lightgbm_grid.best_params_
    }
    
    print("Meilleurs hyperparamètres pour chaque modèle :")
    for model_name, params in best_params.items():
        print(f"{model_name}: {params}")
    
    # Calcul des métriques pour chaque modèle
    models = {
        'DummyClassifier': dummy_clf,
        'LogisticRegression': log_reg_grid,
        'DecisionTreeClassifier': tree_grid,
        'RandomForestClassifier': rf_grid,
        'CatBoostClassifier': catboost_grid,
        'LightGBMClassifier': lightgbm_grid
    }

    metrics = []

    for name, model in models.items():
        train_pred = model.predict(train_final)
        auc_score = roc_auc_score(train_labels, model.predict_proba(train_final)[:, 1])
        precision = precision_score(train_labels, train_pred, zero_division=0)
        recall = recall_score(train_labels, train_pred, zero_division=0)
        f1 = f1_score(train_labels, train_pred, zero_division=0)
        cm = confusion_matrix(train_labels, train_pred)
        cm_str = f"TN: {cm[0, 0]}, FP: {cm[0, 1]}, FN: {cm[1, 0]}, TP: {cm[1, 1]}"
        time_metric = measure_prediction_time(model, test_final)
        metrics.append([
            name,
            f"{auc_score:.5f}",
            f"{precision:.5f}",
            f"{recall:.5f}",
            f"{f1:.5f}",
            cm_str,
            time_metric
        ])
    
    # Affichage des résultats
    headers = ['Model', 'ROC AUC', 'Precision', 'Recall', 'F1 Score', 'Confusion Matrix', 'Time for 1000 Predictions (s)']
    results_table = tabulate(metrics, headers, tablefmt='grid')
    
    print("Résultats des modèles :")
    print(results_table)
    
    return {
        'DummyClassifier': dummy_predictions,
        'LogisticRegression': log_reg_grid.predict_proba(test_final)[:, 1],
        'DecisionTreeClassifier': tree_grid.predict_proba(test_final)[:, 1],
        'RandomForestClassifier': rf_grid.predict_proba(test_final)[:, 1],
        'CatBoostClassifier': catboost_predictions,
        'LightGBMClassifier': lightgbm_predictions
    }

# Appel de la fonction avec les chemins des fichiers
predictions = process_and_predict('data/application_train_split.csv', 'data/application_test_split.csv')


KeyboardInterrupt: 

# Ponderation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
import lightgbm as lgb
import logging
import sys
import time
from tabulate import tabulate

def measure_prediction_time(model, X, n_predictions=1000):
    """Mesurer le temps de prédiction moyen sur n_predictions échantillons."""
    start_time = time.time()
    for _ in range(n_predictions):
        _ = model.predict_proba(X)[:, 1]
    end_time = time.time()
    elapsed_time = end_time - start_time
    return "{:.2f} s".format(elapsed_time)

def process_and_predict(train_file, test_file):
    # Configurer le logging pour ignorer les messages d'information de LightGBM
    logging.getLogger('lightgbm').setLevel(logging.ERROR)
    
    # Rediriger stdout pour éviter les messages d'information de LightGBM
    original_stdout = sys.stdout
    sys.stdout = open('/dev/null', 'w')  # Sur Unix/Linux/Mac
    
    # Charger les données
    app_train = pd.read_csv(train_file)
    app_test = pd.read_csv(test_file)
    
    # Convertir toutes les colonnes en numériques en utilisant LabelEncoder
    label_encoders = {}
    for col in app_train.columns:
        if app_train[col].dtype == 'object':
            le = LabelEncoder()
            app_train[col] = le.fit_transform(app_train[col].astype(str))
            if col in app_test.columns:
                app_test[col] = le.transform(app_test[col].astype(str))
            label_encoders[col] = le
    
    # Séparer les données d'entraînement et les labels
    if 'TARGET' in app_train:
        train = app_train.drop(columns=['TARGET'])
        train_labels = app_train['TARGET']
    else:
        raise ValueError("La colonne 'TARGET' n'existe pas dans les données d'entraînement.")
    
    # Préparer les données de test sans l'identifiant
    test = app_test.drop(columns=['SK_ID_CURR'])
    
    # S'assurer que les colonnes sont alignées avant l'imputation
    common_cols = train.columns.intersection(test.columns)
    train = train[common_cols]
    test = test[common_cols]
    
    # Imputer les valeurs manquantes avec la médiane
    imputer = SimpleImputer(strategy='median')
    train_numeric = imputer.fit_transform(train)
    test_numeric = imputer.transform(test)
    
    # Normaliser les colonnes numériques
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_numeric = scaler.fit_transform(train_numeric)
    test_numeric = scaler.transform(test_numeric)
    
    # Convertir les matrices numpy en DataFrames pandas
    train_final = pd.DataFrame(train_numeric, columns=common_cols)
    test_final = pd.DataFrame(test_numeric, columns=common_cols)
    
    # Créer et entraîner le DummyClassifier
    dummy_clf = DummyClassifier(strategy="stratified")
    dummy_clf.fit(train_final, train_labels)
    dummy_predictions = dummy_clf.predict_proba(test_final)[:, 1]
    dummy_auc = roc_auc_score(train_labels, dummy_clf.predict_proba(train_final)[:, 1])
    dummy_time = measure_prediction_time(dummy_clf, test_final)
    
    # Définir les paramètres pour GridSearchCV pour LogisticRegression avec pondération des classes
    log_reg_params = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
    log_reg = LogisticRegression(max_iter=3000, class_weight='balanced')  # Ajout de class_weight='balanced'
    log_reg_grid = GridSearchCV(log_reg, log_reg_params, cv=5, scoring='roc_auc')
    log_reg_grid.fit(train_final, train_labels)
    log_reg_auc = roc_auc_score(train_labels, log_reg_grid.predict_proba(train_final)[:, 1])
    log_reg_time = measure_prediction_time(log_reg_grid, test_final)
    
    # Définir les paramètres pour GridSearchCV pour DecisionTreeClassifier avec pondération des classes
    tree_params = {'max_depth': [5, 7, 10], 'min_samples_split': [10, 20, 50]}
    decision_tree = DecisionTreeClassifier(class_weight='balanced')  # Ajout de class_weight='balanced'
    tree_grid = GridSearchCV(decision_tree, tree_params, cv=5, scoring='roc_auc')
    tree_grid.fit(train_final, train_labels)
    tree_auc = roc_auc_score(train_labels, tree_grid.predict_proba(train_final)[:, 1])
    tree_time = measure_prediction_time(tree_grid, test_final)
    
    # Définir les paramètres pour GridSearchCV pour RandomForestClassifier
    rf_params = {'n_estimators': [10, 50, 100], 'max_depth': [2,3,4,5, 7,10,15]}
    random_forest = RandomForestClassifier(n_jobs=-1,class_weight='balanced')
    rf_grid = GridSearchCV(random_forest, rf_params, cv=5, scoring='roc_auc')
    rf_grid.fit(train_final, train_labels)
    rf_auc = roc_auc_score(train_labels, rf_grid.predict_proba(train_final)[:, 1])
    rf_time = measure_prediction_time(rf_grid, test_final)
    
    # Définir les paramètres pour GridSearchCV pour CatBoostClassifier
    catboost_params = {'depth': [1,2,3,4,5], 'learning_rate': [0.01, 0.1, 0.3]}
    catboost_model = CatBoostClassifier(iterations=100, loss_function='Logloss', verbose=0)
    catboost_grid = GridSearchCV(catboost_model, catboost_params, cv=5, scoring='roc_auc')
    catboost_grid.fit(train_final, train_labels)
    catboost_auc = roc_auc_score(train_labels, catboost_grid.predict_proba(train_final)[:, 1])
    catboost_predictions = catboost_grid.predict_proba(test_final)[:, 1]
    catboost_time = measure_prediction_time(catboost_grid, test_final)
    
    # Définir les paramètres pour GridSearchCV pour LightGBMClassifier
    lightgbm_params = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [1,2,3,4,5],
        'num_leaves': [2**6 - 1, 2**7 - 1, 2**8 - 1]  # Ajusté pour respecter la condition
    }
    lightgbm_model = lgb.LGBMClassifier()
    lightgbm_grid = GridSearchCV(lightgbm_model, lightgbm_params, cv=5, scoring='roc_auc')
    lightgbm_grid.fit(train_final, train_labels)
    lightgbm_auc = roc_auc_score(train_labels, lightgbm_grid.predict_proba(train_final)[:, 1])
    lightgbm_predictions = lightgbm_grid.predict_proba(test_final)[:, 1]
    lightgbm_time = measure_prediction_time(lightgbm_grid, test_final)
    
    # Rétablir stdout
    sys.stdout = original_stdout

    # Afficher les meilleurs hyperparamètres pour chaque modèle
    best_params = {
        'LogisticRegression': log_reg_grid.best_params_,
        'DecisionTreeClassifier': tree_grid.best_params_,
        'RandomForestClassifier': rf_grid.best_params_,
        'CatBoostClassifier': catboost_grid.best_params_,
        'LightGBMClassifier': lightgbm_grid.best_params_
    }
    
    print("Meilleurs hyperparamètres pour chaque modèle :")
    for model_name, params in best_params.items():
        print(f"{model_name}: {params}")
    
    # Calcul des métriques pour chaque modèle
    models = {
        'DummyClassifier': dummy_clf,
        'LogisticRegression': log_reg_grid,
        'DecisionTreeClassifier': tree_grid,
        'RandomForestClassifier': rf_grid,
        'CatBoostClassifier': catboost_grid,
        'LightGBMClassifier': lightgbm_grid
    }

    metrics = []

    for name, model in models.items():
        train_pred = model.predict(train_final)
        auc_score = roc_auc_score(train_labels, model.predict_proba(train_final)[:, 1])
        precision = precision_score(train_labels, train_pred, zero_division=0)
        recall = recall_score(train_labels, train_pred, zero_division=0)
        f1 = f1_score(train_labels, train_pred, zero_division=0)
        cm = confusion_matrix(train_labels, train_pred)
        cm_str = f"TN: {cm[0, 0]}, FP: {cm[0, 1]}, FN: {cm[1, 0]}, TP: {cm[1, 1]}"
        time_metric = measure_prediction_time(model, test_final)
        metrics.append([
            name,
            f"{auc_score:.5f}",
            f"{precision:.5f}",
            f"{recall:.5f}",
            f"{f1:.5f}",
            cm_str,
            time_metric
        ])
    
    # Affichage des résultats
    headers = ['Model', 'ROC AUC', 'Precision', 'Recall', 'F1 Score', 'Confusion Matrix', 'Time for 1000 Predictions (s)']
    results_table = tabulate(metrics, headers, tablefmt='grid')
    
    print("Résultats des modèles :")
    print(results_table)
    
    return {
        'DummyClassifier': dummy_predictions,
        'LogisticRegression': log_reg_grid.predict_proba(test_final)[:, 1],
        'DecisionTreeClassifier': tree_grid.predict_proba(test_final)[:, 1],
        'RandomForestClassifier': rf_grid.predict_proba(test_final)[:, 1],
        'CatBoostClassifier': catboost_predictions,
        'LightGBMClassifier': lightgbm_predictions
    }

# Appel de la fonction avec les chemins des fichiers
predictions = process_and_predict('data/application_train_split.csv', 'data/application_test_split.csv')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
import lightgbm as lgb
import logging
import sys
import time
from tabulate import tabulate

def measure_prediction_time(model, X, n_predictions=1000):
    """Mesurer le temps de prédiction moyen sur n_predictions échantillons."""
    start_time = time.time()
    for _ in range(n_predictions):
        _ = model.predict_proba(X)[:, 1]
    end_time = time.time()
    elapsed_time = end_time - start_time
    return "{:.2f}".format(elapsed_time)

def calculate_class_weights(labels):
    """Calculer les poids des classes basés sur la distribution des labels."""
    class_counts = labels.value_counts().to_dict()
    total = sum(class_counts.values())
    class_weights = {cls: total / (len(class_counts) * count) for cls, count in class_counts.items()}
    return class_weights

def evaluate_model(model, X_train, y_train, X_test, y_test):
    """Évaluer les performances d'un modèle sur les ensembles d'entraînement et de test."""
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    train_proba = model.predict_proba(X_train)[:, 1]
    test_proba = model.predict_proba(X_test)[:, 1]
    
    metrics = {
        'train_auc': roc_auc_score(y_train, train_proba),
        'test_auc': roc_auc_score(y_test, test_proba),
        'train_precision': precision_score(y_train, train_pred, zero_division=0),
        'test_precision': precision_score(y_test, test_pred, zero_division=0),
        'train_recall': recall_score(y_train, train_pred, zero_division=0),
        'test_recall': recall_score(y_test, test_pred, zero_division=0),
        'train_f1': f1_score(y_train, train_pred, zero_division=0),
        'test_f1': f1_score(y_test, test_pred, zero_division=0),
        'train_cm': confusion_matrix(y_train, train_pred),
        'test_cm': confusion_matrix(y_test, test_pred)
    }
    
    return metrics

def process_and_predict(train_file, test_file):
    # Configurer le logging pour ignorer les messages d'information de LightGBM
    logging.getLogger('lightgbm').setLevel(logging.ERROR)
    
    # Rediriger stdout pour éviter les messages d'information de LightGBM
    original_stdout = sys.stdout
    sys.stdout = open('/dev/null', 'w')  # Sur Unix/Linux/Mac
    
    # Charger les données
    app_train = pd.read_csv(train_file)
    app_test = pd.read_csv(test_file)
    
    # Convertir toutes les colonnes en numériques en utilisant LabelEncoder
    label_encoders = {}
    for col in app_train.columns:
        if app_train[col].dtype == 'object':
            le = LabelEncoder()
            app_train[col] = le.fit_transform(app_train[col].astype(str))
            if col in app_test.columns:
                app_test[col] = le.transform(app_test[col].astype(str))
            label_encoders[col] = le
    
    # Séparer les données d'entraînement et les labels
    if 'TARGET' in app_train:
        train = app_train.drop(columns=['TARGET'])
        train_labels = app_train['TARGET']
    else:
        raise ValueError("La colonne 'TARGET' n'existe pas dans les données d'entraînement.")
    
    # Préparer les données de test sans l'identifiant
    test = app_test.drop(columns=['SK_ID_CURR'])
    
    # S'assurer que les colonnes sont alignées avant l'imputation
    common_cols = train.columns.intersection(test.columns)
    train = train[common_cols]
    test = test[common_cols]
    
    # Imputer les valeurs manquantes avec la médiane
    imputer = SimpleImputer(strategy='median')
    train_numeric = imputer.fit_transform(train)
    test_numeric = imputer.transform(test)
    
    # Normaliser les colonnes numériques
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_numeric = scaler.fit_transform(train_numeric)
    test_numeric = scaler.transform(test_numeric)
    
    # Convertir les matrices numpy en DataFrames pandas
    train_final = pd.DataFrame(train_numeric, columns=common_cols)
    test_final = pd.DataFrame(test_numeric, columns=common_cols)
    
    # Créer et entraîner les modèles
    models = {}
    
    # DummyClassifier
    dummy_clf = DummyClassifier(strategy="stratified")
    dummy_clf.fit(train_final, train_labels)
    models['DummyClassifier'] = dummy_clf

    # LogisticRegression
    log_reg_params = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
    log_reg = LogisticRegression(max_iter=3000, class_weight='balanced')
    log_reg_grid = GridSearchCV(log_reg, log_reg_params, cv=5, scoring='roc_auc')
    log_reg_grid.fit(train_final, train_labels)
    models['LogisticRegression'] = log_reg_grid
    
    # DecisionTreeClassifier
    tree_params = {'max_depth': [5,6, 7,8, 10], 'min_samples_split': [10, 20, 50]}
    decision_tree = DecisionTreeClassifier(class_weight='balanced')
    tree_grid = GridSearchCV(decision_tree, tree_params, cv=5, scoring='roc_auc')
    tree_grid.fit(train_final, train_labels)
    models['DecisionTreeClassifier'] = tree_grid
    
    # RandomForestClassifier
    rf_params = {'n_estimators': [10, 50, 100], 'max_depth': [2, 3, 4, 5,6, 7,8, 10]}
    random_forest = RandomForestClassifier(n_jobs=-1, class_weight='balanced')
    rf_grid = GridSearchCV(random_forest, rf_params, cv=5, scoring='roc_auc')
    rf_grid.fit(train_final, train_labels)
    models['RandomForestClassifier'] = rf_grid
    
    # CatBoostClassifier
    class_weights = calculate_class_weights(train_labels)
    catboost_params = {'depth': [1, 2, 3, 4, 5], 'learning_rate': [0.01, 0.1,0.2, 0.3]}
    catboost_model = CatBoostClassifier(iterations=100, loss_function='Logloss', class_weights=class_weights, verbose=0)
    catboost_grid = GridSearchCV(catboost_model, catboost_params, cv=5, scoring='roc_auc')
    catboost_grid.fit(train_final, train_labels)
    models['CatBoostClassifier'] = catboost_grid
    
    # LightGBMClassifier
    lightgbm_params = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1,0.2, 0.3],
        'max_depth': [1, 2, 3, 4, 5],
        'num_leaves': [2**6 - 1, 2**7 - 1, 2**8 - 1]
    }
    lightgbm_model = lgb.LGBMClassifier(class_weight=class_weights)
    lightgbm_grid = GridSearchCV(lightgbm_model, lightgbm_params, cv=5, scoring='roc_auc')
    lightgbm_grid.fit(train_final, train_labels)
    models['LightGBMClassifier'] = lightgbm_grid
    
    # Rétablir stdout
    sys.stdout = original_stdout

    # Afficher les meilleurs hyperparamètres pour chaque modèle
    best_params = {name: model.best_params_ if hasattr(model, 'best_params_') else {} for name, model in models.items()}
    
    print("Meilleurs hyperparamètres pour chaque modèle :")
    for model_name, params in best_params.items():
        print(f"{model_name}: {params}")
    
    # Préparer les métriques pour les données d'entraînement et de test
    metrics_train = []
    metrics_test = []

    catboost_predictions = None
    
    for name, model in models.items():
        metrics = evaluate_model(model, train_final, train_labels, test_final, app_test['TARGET'])
        train_cm = metrics['train_cm']
        test_cm = metrics['test_cm']
        metrics_train.append([
            name,
            f"{metrics['train_auc']:.5f}",
            f"{metrics['train_precision']:.5f}",
            f"{metrics['train_recall']:.5f}",
            f"{metrics['train_f1']:.5f}",
            f"{train_cm[0,0]} | {train_cm[0,1]}\n{train_cm[1,0]} | {train_cm[1,1]}",
            measure_prediction_time(model, test_final)
        ])
        metrics_test.append([
            name,
            f"{metrics['test_auc']:.5f}",
            f"{metrics['test_precision']:.5f}",
            f"{metrics['test_recall']:.5f}",
            f"{metrics['test_f1']:.5f}",
            f"{test_cm[0,0]} | {test_cm[0,1]}\n{test_cm[1,0]} | {test_cm[1,1]}",
            measure_prediction_time(model, test_final)
        ])
        
        # Spécial pour CatBoostClassifier
        if name == 'CatBoostClassifier':
            catboost_predictions = model.predict_proba(test_final)[:, 1]
    
    # Affichage des résultats pour les données d'entraînement
    headers_train = ['Model', 'AUC', 'Precision', 'Recall', 'F1-Score', 'Confusion Matrix', 'Time (1000 estimations)']
    results_table_train = tabulate(metrics_train, headers_train, tablefmt='pipe')
    
    print("\nMétriques pour les données d'entraînement :")
    print(results_table_train)
    
    # Affichage des résultats pour les données de test
    headers_test = ['Model', 'AUC', 'Precision', 'Recall', 'F1-Score', 'Confusion Matrix', 'Time (1000 estimations)']
    results_table_test = tabulate(metrics_test, headers_test, tablefmt='pipe')
    
    print("\nMétriques pour les données de test :")
    print(results_table_test)
    
    return best_params, metrics_test, catboost_predictions

# Appel de la fonction avec les chemins des fichiers
predictions = process_and_predict('data/application_train_split.csv', 'data/application_test_split.csv')


# Separation des fonctions 

## Preparation

In [1043]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer

def clean_data(train_file, test_file):
    """Nettoyer et préparer les données pour l'entraînement et le test."""
    print("Début du nettoyage des données...")
    
    # Charger les données
    app_train = pd.read_csv(train_file)
    app_test = pd.read_csv(test_file)
    
    # Convertir toutes les colonnes en numériques en utilisant LabelEncoder
    label_encoders = {}
    for col in app_train.columns:
        if app_train[col].dtype == 'object':
            le = LabelEncoder()
            app_train[col] = le.fit_transform(app_train[col].astype(str))
            if col in app_test.columns:
                app_test[col] = le.transform(app_test[col].astype(str))
            label_encoders[col] = le
    
    # Séparer les données d'entraînement et les labels
    if 'TARGET' in app_train:
        train = app_train.drop(columns=['TARGET'])
        train_labels = app_train['TARGET']
    else:
        raise ValueError("La colonne 'TARGET' n'existe pas dans les données d'entraînement.")
    
    # Préparer les données de test sans l'identifiant
    test = app_test.drop(columns=['SK_ID_CURR'])
    
    # S'assurer que les colonnes sont alignées avant l'imputation
    common_cols = train.columns.intersection(test.columns)
    train = train[common_cols]
    test = test[common_cols]
    
    # Imputer les valeurs manquantes avec la médiane
    imputer = SimpleImputer(strategy='median')
    train_numeric = imputer.fit_transform(train)
    test_numeric = imputer.transform(test)
    
    # Normaliser les colonnes numériques
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_numeric = scaler.fit_transform(train_numeric)
    test_numeric = scaler.transform(test_numeric)
    
    # Convertir les matrices numpy en DataFrames pandas
    train_final = pd.DataFrame(train_numeric, columns=common_cols)
    test_final = pd.DataFrame(test_numeric, columns=common_cols)
    
    print("Fin du nettoyage des données.")
    return train_final, test_final, train_labels, app_test['TARGET']


## Suppression Output

In [1046]:
import sys
import os

class SuppressOutput:
    """Context manager to suppress stdout and stderr."""
    def __enter__(self):
        self._original_stdout = sys.stdout
        self._original_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, 'w')
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout.close()
        sys.stderr.close()
        sys.stdout = self._original_stdout
        sys.stderr = self._original_stderr

## Ponderation

In [1049]:
def calculate_class_weights(y):
    """Calculer les poids des classes pour un dataset déséquilibré."""
    print("Calcul de la pondération des classes...")
    from sklearn.utils.class_weight import compute_class_weight
    
    classes = np.unique(y)
    class_weights = compute_class_weight('balanced', classes=classes, y=y)
    class_weight_dict = dict(zip(classes, class_weights))
    
    print(f"Pondération des classes: {class_weight_dict}")
    return class_weight_dict


## Recherche des HP

In [1052]:
from sklearn.model_selection import GridSearchCV

def search_hyperparameters(train_final, train_labels):
    """Rechercher les meilleurs hyperparamètres pour chaque modèle."""
    print("Début de la recherche des hyperparamètres...")
    
    models = {}
    
    # LogisticRegression
    log_reg_params = {'C': [1,0.1,10],'solver': ['newton-cg', 'lbfgs'],}
    log_reg = LogisticRegression(max_iter=2500, class_weight='balanced')
    log_reg_grid = GridSearchCV(log_reg, log_reg_params, cv=5, scoring='roc_auc')
    log_reg_grid.fit(train_final, train_labels)
    models['LogisticRegression'] = log_reg_grid.best_params_
    
    # DecisionTreeClassifier
    tree_params = {'max_depth': [4,6,7], 'min_samples_split': [3, 4, 2]}
    decision_tree = DecisionTreeClassifier(class_weight='balanced')
    tree_grid = GridSearchCV(decision_tree, tree_params, cv=5, scoring='roc_auc')
    tree_grid.fit(train_final, train_labels)
    models['DecisionTreeClassifier'] = tree_grid.best_params_
    
    # RandomForestClassifier
    rf_params = {'n_estimators': [300], 'max_depth': [5, 10,15]}
    random_forest = RandomForestClassifier(n_jobs=-1, class_weight='balanced')
    rf_grid = GridSearchCV(random_forest, rf_params, cv=5, scoring='roc_auc')
    rf_grid.fit(train_final, train_labels)
    models['RandomForestClassifier'] = rf_grid.best_params_
    
    # CatBoostClassifier
    class_weights = calculate_class_weights(train_labels)
    catboost_params = {'depth': [2, 3], 'learning_rate': [0.01, 0.1,], 'l2_leaf_reg': [5,6],'bagging_temperature': [0.3, 0.2],'loss_function': ['Logloss']}
    catboost_model = CatBoostClassifier(iterations=500,auto_class_weights='Balanced', verbose=0)
    catboost_grid = GridSearchCV(catboost_model, catboost_params, cv=5, scoring='roc_auc')
    catboost_grid.fit(train_final, train_labels)
    models['CatBoostClassifier'] = catboost_grid.best_params_
    
    # LightGBMClassifier
    lightgbm_params = {
        'n_estimators': [450],
        'learning_rate': [0.1, 0.2],
        'max_depth': [2, 3,],
        'num_leaves': [2**6 - 1, 2**7 - 1, 2**8 - 1],
        'verbosity': [-1]
    }
    lightgbm_model = lgb.LGBMClassifier()
    lightgbm_grid = GridSearchCV(lightgbm_model, lightgbm_params, cv=5, scoring='roc_auc')
    lightgbm_grid.fit(train_final, train_labels)
    models['LightGBMClassifier'] = lightgbm_grid.best_params_
    
    print("Fin de la recherche des hyperparamètres.")
    return models


## Entrainement

In [1055]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb

def train_models(train_final, train_labels, best_params):
    """Entraîne les modèles avec les meilleurs hyperparamètres trouvés et affiche les meilleurs paramètres."""
    print("Début de l'entraînement des modèles...")
    
    models = {}
    
    # Logistic Regression
    print("\nEntraînement du modèle Logistic Regression...")
    log_reg_model = LogisticRegression(class_weight='balanced', max_iter=3000, **best_params['LogisticRegression'])
    log_reg_model.fit(train_final, train_labels)
    models['LogisticRegression'] = log_reg_model
    print("Meilleurs paramètres pour Logistic Regression :")
    print(best_params['LogisticRegression'])
    
    # Decision Tree Classifier
    print("\nEntraînement du modèle Decision Tree Classifier...")
    decision_tree_model = DecisionTreeClassifier(class_weight='balanced', **best_params['DecisionTreeClassifier'])
    decision_tree_model.fit(train_final, train_labels)
    models['DecisionTreeClassifier'] = decision_tree_model
    print("Meilleurs paramètres pour Decision Tree Classifier :")
    print(best_params['DecisionTreeClassifier'])
    
    # Random Forest Classifier
    print("\nEntraînement du modèle Random Forest Classifier...")
    random_forest_model = RandomForestClassifier(n_jobs=-1, class_weight='balanced', **best_params['RandomForestClassifier'])
    random_forest_model.fit(train_final, train_labels)
    models['RandomForestClassifier'] = random_forest_model
    print("Meilleurs paramètres pour Random Forest Classifier :")
    print(best_params['RandomForestClassifier'])
    
    # CatBoost Classifier
    print("\nEntraînement du modèle CatBoost Classifier...")
    catboost_model = CatBoostClassifier(auto_class_weights='Balanced',silent=True, **best_params['CatBoostClassifier'])
    catboost_model.fit(train_final, train_labels)
    models['CatBoostClassifier'] = catboost_model
    print("Meilleurs paramètres pour CatBoost Classifier :")
    print(best_params['CatBoostClassifier'])
    
    # LightGBM Classifier
    print("\nEntraînement du modèle LightGBM...")
    lightgbm_params = best_params['LightGBMClassifier']
    lightgbm_params['verbosity'] = -1  # S'assurer que verbosity est désactivé
    lightgbm_model = lgb.LGBMClassifier(class_weight='balanced', **lightgbm_params)
    lightgbm_model.fit(train_final, train_labels)
    models['LightGBMClassifier'] = lightgbm_model
    print("Meilleurs paramètres pour LightGBM Classifier :")
    print(lightgbm_params)
    
    print("\nFin de l'entraînement des modèles.")
    return models


## Evaluation des perfs

In [1058]:
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score, confusion_matrix
import time
from tabulate import tabulate

def measure_prediction_time(model, X, n_estimations=1000):
    """Mesurer le temps de prédiction pour un nombre donné d'estimations.
    
    Args:
        model: Le modèle à évaluer.
        X: Les données d'entrée pour la prédiction.
        n_estimations: Le nombre d'estimations à faire (par défaut 1000).

    Returns:
        avg_time: Temps moyen de prédiction pour une estimation, en secondes.
        total_time: Temps total pour les 1000 prédictions, en secondes.
    """
    start_time = time.time()
    for _ in range(n_estimations):
        model.predict(X)
    end_time = time.time()
    total_time = end_time - start_time
    return total_time


def evaluate_models(models, X_train, y_train, X_test, y_test):
    """Évaluer les performances des modèles sur les ensembles d'entraînement et de test."""
    print("Début de l'évaluation des modèles...")
    
    metrics_train = []
    metrics_test = []
    
    for name, model in models.items():
        if hasattr(model, 'predict'):
            train_pred = model.predict(X_train)
            test_pred = model.predict(X_test)
            train_proba = model.predict_proba(X_train)[:, 1]
            test_proba = model.predict_proba(X_test)[:, 1]
        else:
            train_pred = model.best_estimator_.predict(X_train)
            test_pred = model.best_estimator_.predict(X_test)
            train_proba = model.best_estimator_.predict_proba(X_train)[:, 1]
            test_proba = model.best_estimator_.predict_proba(X_test)[:, 1]
        
        train_auc = roc_auc_score(y_train, train_proba)
        test_auc = roc_auc_score(y_test, test_proba)
        train_precision = precision_score(y_train, train_pred, zero_division=0)
        test_precision = precision_score(y_test, test_pred, zero_division=0)
        train_recall = recall_score(y_train, train_pred, zero_division=0)
        test_recall = recall_score(y_test, test_pred, zero_division=0)
        train_f1 = f1_score(y_train, train_pred, zero_division=0)
        test_f1 = f1_score(y_test, test_pred, zero_division=0)
        train_cm = confusion_matrix(y_train, train_pred)
        test_cm = confusion_matrix(y_test, test_pred)
        
        metrics_train.append([
            name,
            f"{train_auc:.5f}",
            f"{train_precision:.5f}",
            f"{train_recall:.5f}",
            f"{train_f1:.5f}",
            f"{train_cm[0,0]} | {train_cm[0,1]}\n{train_cm[1,0]} | {train_cm[1,1]}",
            measure_prediction_time(model, X_test)
        ])
        
        metrics_test.append([
            name,
            f"{test_auc:.5f}",
            f"{test_precision:.5f}",
            f"{test_recall:.5f}",
            f"{test_f1:.5f}",
            f"{test_cm[0,0]} | {test_cm[0,1]}\n{test_cm[1,0]} | {test_cm[1,1]}",
            measure_prediction_time(model, X_test)
        ])
    
    print("Fin de l'évaluation des modèles.")
    return metrics_train, metrics_test


## Fonction principale

In [1061]:
def process_and_predict(train_file, test_file):
    """Fonction principale pour nettoyer les données, rechercher les hyperparamètres,
    entraîner les modèles, et évaluer les performances."""
    print("Début du traitement et des prédictions...")
    
    # Nettoyage des données
    train_final, test_final, train_labels, test_labels = clean_data(train_file, test_file)
    
    # Recherche des hyperparamètres
    best_params = search_hyperparameters(train_final, train_labels)
    
    # Entraînement des modèles
    models = train_models(train_final, train_labels, best_params)
    
    # Évaluation des performances
    metrics_train, metrics_test = evaluate_models(models, train_final, train_labels, test_final, test_labels)
    
    # Affichage des résultats pour les données d'entraînement
    headers_train = ['Model', 'ROC AUC', 'Precision', 'Recall', 'F1-Score', 'Confusion Matrix', 'Time (1000 estimations)']
    results_table_train = tabulate(metrics_train, headers_train, tablefmt='pipe')
    
    print("\nMétriques pour les données d'entraînement :")
    print(results_table_train)
    
    # Affichage des résultats pour les données de test
    headers_test = ['Model', 'ROC AUC', 'Precision', 'Recall', 'F1-Score', 'Confusion Matrix', 'Time (1000 estimations)']
    results_table_test = tabulate(metrics_test, headers_test, tablefmt='pipe')
    
    print("\nMétriques pour les données de test :")
    print(results_table_test)
    
    print("Fin du traitement et des prédictions.")
    return metrics_train, metrics_test


## Appel 

In [1064]:
# Appel de la fonction principale avec les chemins des fichiers
metrics_train, metrics_test = process_and_predict('data/application_train_split.csv', 'data/application_test_split.csv')


Début du traitement et des prédictions...
Début du nettoyage des données...
Fin du nettoyage des données.
Début de la recherche des hyperparamètres...
Calcul de la pondération des classes...
Pondération des classes: {0: 0.5436743868865709, 1: 6.224178811010872}
Fin de la recherche des hyperparamètres.
Début de l'entraînement des modèles...

Entraînement du modèle Logistic Regression...
Meilleurs paramètres pour Logistic Regression :
{'C': 10, 'solver': 'newton-cg'}

Entraînement du modèle Decision Tree Classifier...
Meilleurs paramètres pour Decision Tree Classifier :
{'max_depth': 6, 'min_samples_split': 3}

Entraînement du modèle Random Forest Classifier...
Meilleurs paramètres pour Random Forest Classifier :
{'max_depth': 10, 'n_estimators': 300}

Entraînement du modèle CatBoost Classifier...
Meilleurs paramètres pour CatBoost Classifier :
{'bagging_temperature': 0.3, 'depth': 3, 'l2_leaf_reg': 6, 'learning_rate': 0.1, 'loss_function': 'Logloss'}

Entraînement du modèle LightGBM...


# Avec feature engineering

## Nettoyage

In [293]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer

def clean_data(train_file, test_file):
    """Nettoyer et préparer les données pour l'entraînement et le test."""
    print("Début du nettoyage des données...")
    
    # Charger les données
    app_train = pd.read_csv(train_file)
    app_test = pd.read_csv(test_file)
    
    # Convertir toutes les colonnes en numériques en utilisant LabelEncoder
    label_encoders = {}
    for col in app_train.columns:
        if app_train[col].dtype == 'object':
            le = LabelEncoder()
            app_train[col] = le.fit_transform(app_train[col].astype(str))
            if col in app_test.columns:
                app_test[col] = le.transform(app_test[col].astype(str))
            label_encoders[col] = le
    
    # Appliquer l'ingénierie des fonctionnalités
    app_train, app_test = feature_engineering(app_train, app_test)
    
    # Séparer les données d'entraînement et les labels
    if 'TARGET' in app_train:
        train = app_train.drop(columns=['TARGET'])
        train_labels = app_train['TARGET']
    else:
        raise ValueError("La colonne 'TARGET' n'existe pas dans les données d'entraînement.")
    
    # Préparer les données de test sans l'identifiant
    test = app_test.drop(columns=['SK_ID_CURR'])
    
    # S'assurer que les colonnes sont alignées avant l'imputation
    common_cols = train.columns.intersection(test.columns)
    train = train[common_cols]
    test = test[common_cols]
    
    # Imputer les valeurs manquantes avec la médiane
    imputer = SimpleImputer(strategy='median')
    train_numeric = imputer.fit_transform(train)
    test_numeric = imputer.transform(test)
    
    # Normaliser les colonnes numériques
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_numeric = scaler.fit_transform(train_numeric)
    test_numeric = scaler.transform(test_numeric)
    
    # Convertir les matrices numpy en DataFrames pandas
    train_final = pd.DataFrame(train_numeric, columns=common_cols)
    test_final = pd.DataFrame(test_numeric, columns=common_cols)
    
    print("Fin du nettoyage des données.")
    return train_final, test_final, train_labels, app_test['TARGET']


## feature engineering

In [296]:
def feature_engineering(app_train, app_test):
    """Créer de nouvelles fonctionnalités à partir des données d'entraînement et de test."""
    # Création de nouvelles fonctionnalités
    app_train['annuity_income_ratio'] = app_train['AMT_ANNUITY'] / app_train['AMT_INCOME_TOTAL']
    app_test['annuity_income_ratio'] = app_test['AMT_ANNUITY'] / app_test['AMT_INCOME_TOTAL']
    
    app_train['credit_annuity_ratio'] = app_train['AMT_CREDIT'] / app_train['AMT_ANNUITY']
    app_test['credit_annuity_ratio'] = app_test['AMT_CREDIT'] / app_test['AMT_ANNUITY']
    
    app_train['credit_goods_price_ratio'] = app_train['AMT_CREDIT'] / app_train['AMT_GOODS_PRICE']
    app_test['credit_goods_price_ratio'] = app_test['AMT_CREDIT'] / app_test['AMT_GOODS_PRICE']
    
    app_train['credit_downpayment'] = app_train['AMT_GOODS_PRICE'] - app_train['AMT_CREDIT']
    app_test['credit_downpayment'] = app_test['AMT_GOODS_PRICE'] - app_test['AMT_CREDIT']
    
    app_train['AGE_INT'] = (app_train['DAYS_BIRTH'] / -365).astype(int)
    app_test['AGE_INT'] = (app_test['DAYS_BIRTH'] / -365).astype(int)
    
    return app_train, app_test


## Recherche des Hyperparamètres

In [299]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

def search_hyperparameters(train_final, train_labels):
    """Rechercher les meilleurs hyperparamètres pour les modèles CatBoost et LogisticRegression."""
    print("Début de la recherche des hyperparamètres...")
    
    models = {}
    
    # LogisticRegression
    log_reg_params = {'C': [1, 0.1, 10], 'solver': ['newton-cg', 'lbfgs'],'penalty': ['l2']}
    log_reg = LogisticRegression(max_iter=5000, class_weight='balanced')
    log_reg_grid = GridSearchCV(log_reg, log_reg_params, cv=5, scoring='roc_auc')
    log_reg_grid.fit(train_final, train_labels)
    models['LogisticRegression'] = log_reg_grid.best_params_
    
    # CatBoostClassifier
    catboost_params = {
        'depth': [2, 3],
        'learning_rate': [0.01, 0.1],
        'l2_leaf_reg': [5, 6],
        'bagging_temperature': [0.3, 0.2],
        'loss_function': ['Logloss']
    }
    catboost_model = CatBoostClassifier(iterations=500, auto_class_weights='Balanced', verbose=0)
    catboost_grid = GridSearchCV(catboost_model, catboost_params, cv=5, scoring='roc_auc')
    catboost_grid.fit(train_final, train_labels)
    models['CatBoostClassifier'] = catboost_grid.best_params_
    
    print("Fin de la recherche des hyperparamètres.")
    return models


## Entraînement des Modèles

In [302]:
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

def train_models(train_final, train_labels, best_params):
    """Entraîne les modèles CatBoost et LogisticRegression avec les meilleurs hyperparamètres trouvés et affiche les meilleurs paramètres."""
    print("Début de l'entraînement des modèles...")
    
    models = {}
    
    # Logistic Regression
    print("\nEntraînement du modèle Logistic Regression...")
    log_reg_model = LogisticRegression(class_weight='balanced', max_iter=6000, **best_params['LogisticRegression'])
    log_reg_model.fit(train_final, train_labels)
    models['LogisticRegression'] = log_reg_model
    print("Meilleurs paramètres pour Logistic Regression :")
    print(best_params['LogisticRegression'])
    
    # CatBoost Classifier
    print("\nEntraînement du modèle CatBoost Classifier...")
    catboost_model = CatBoostClassifier(auto_class_weights='Balanced', silent=True, **best_params['CatBoostClassifier'])
    catboost_model.fit(train_final, train_labels)
    models['CatBoostClassifier'] = catboost_model
    print("Meilleurs paramètres pour CatBoost Classifier :")
    print(best_params['CatBoostClassifier'])
    
    print("\nFin de l'entraînement des modèles.")
    return models


## Évaluation des Modèles

In [305]:
import time

def measure_prediction_time(model, X, n_estimations=1000):
    """Mesurer le temps de prédiction pour un nombre donné d'estimations."""
    start_time = time.time()
    for _ in range(n_estimations):
        model.predict(X)
    end_time = time.time()
    total_time = end_time - start_time
    return total_time


from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score, confusion_matrix
from tabulate import tabulate

def evaluate_models(models, X_train, y_train, X_test, y_test):
    """Évaluer les performances des modèles CatBoost et LogisticRegression sur les ensembles d'entraînement et de test."""
    print("Début de l'évaluation des modèles...")
    
    metrics_train = []
    metrics_test = []
    
    for name, model in models.items():
        if hasattr(model, 'predict'):
            train_pred = model.predict(X_train)
            test_pred = model.predict(X_test)
            train_proba = model.predict_proba(X_train)[:, 1]
            test_proba = model.predict_proba(X_test)[:, 1]
        else:
            train_pred = model.best_estimator_.predict(X_train)
            test_pred = model.best_estimator_.predict(X_test)
            train_proba = model.best_estimator_.predict_proba(X_train)[:, 1]
            test_proba = model.best_estimator_.predict_proba(X_test)[:, 1]
        
        train_auc = roc_auc_score(y_train, train_proba)
        test_auc = roc_auc_score(y_test, test_proba)
        train_precision = precision_score(y_train, train_pred, zero_division=0)
        test_precision = precision_score(y_test, test_pred, zero_division=0)
        train_recall = recall_score(y_train, train_pred, zero_division=0)
        test_recall = recall_score(y_test, test_pred, zero_division=0)
        train_f1 = f1_score(y_train, train_pred, zero_division=0)
        test_f1 = f1_score(y_test, test_pred, zero_division=0)
        train_cm = confusion_matrix(y_train, train_pred)
        test_cm = confusion_matrix(y_test, test_pred)
        
        metrics_train.append([
            name,
            f"{train_auc:.5f}",
            f"{train_precision:.5f}",
            f"{train_recall:.5f}",
            f"{train_f1:.5f}",
            f"{train_cm[0,0]} | {train_cm[0,1]}\n{train_cm[1,0]} | {train_cm[1,1]}",
            measure_prediction_time(model, X_test)
        ])
        
        metrics_test.append([
            name,
            f"{test_auc:.5f}",
            f"{test_precision:.5f}",
            f"{test_recall:.5f}",
            f"{test_f1:.5f}",
            f"{test_cm[0,0]} | {test_cm[0,1]}\n{test_cm[1,0]} | {test_cm[1,1]}",
            measure_prediction_time(model, X_test)
        ])
    
    print("Fin de l'évaluation des modèles.")
    return metrics_train, metrics_test


## Fonction Principale

In [308]:
def process_and_predict(train_file, test_file):
    """Fonction principale pour nettoyer les données, rechercher les hyperparamètres,
    entraîner les modèles, et évaluer les performances."""
    print("Début du traitement et des prédictions...")
    
    # Nettoyage des données
    train_final, test_final, train_labels, test_labels = clean_data(train_file, test_file)
    
    # Recherche des hyperparamètres
    best_params = search_hyperparameters(train_final, train_labels)
    
    # Entraînement des modèles
    models = train_models(train_final, train_labels, best_params)
    
    # Évaluation des performances
    metrics_train, metrics_test = evaluate_models(models, train_final, train_labels, test_final, test_labels)
    
    # Affichage des résultats pour les données d'entraînement
    headers_train = ['Model', 'ROC AUC', 'Precision', 'Recall', 'F1-Score', 'Confusion Matrix', 'Time (1000 estimations)']
    results_table_train = tabulate(metrics_train, headers_train, tablefmt='pipe')
    
    print("\nMétriques pour les données d'entraînement :")
    print(results_table_train)
    
    # Affichage des résultats pour les données de test
    headers_test = ['Model', 'ROC AUC', 'Precision', 'Recall', 'F1-Score', 'Confusion Matrix', 'Time (1000 estimations)']
    results_table_test = tabulate(metrics_test, headers_test, tablefmt='pipe')
    
    print("\nMétriques pour les données de test :")
    print(results_table_test)
    
    print("Fin du traitement et des prédictions.")
    return metrics_train, metrics_test


## Appel

In [311]:
# Appel de la fonction principale avec les chemins des fichiers
metrics_train, metrics_test = process_and_predict('data/application_train_split.csv', 'data/application_test_split.csv')


Début du traitement et des prédictions...
Début du nettoyage des données...
Fin du nettoyage des données.
Début de la recherche des hyperparamètres...


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


Fin de la recherche des hyperparamètres.
Début de l'entraînement des modèles...

Entraînement du modèle Logistic Regression...
Meilleurs paramètres pour Logistic Regression :
{'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}

Entraînement du modèle CatBoost Classifier...
Meilleurs paramètres pour CatBoost Classifier :
{'bagging_temperature': 0.3, 'depth': 3, 'l2_leaf_reg': 6, 'learning_rate': 0.1, 'loss_function': 'Logloss'}

Fin de l'entraînement des modèles.
Début de l'évaluation des modèles...
Fin de l'évaluation des modèles.

Métriques pour les données d'entraînement :
| Model              |   ROC AUC |   Precision |   Recall |   F1-Score | Confusion Matrix   |   Time (1000 estimations) |
|:-------------------|----------:|------------:|---------:|-----------:|:-------------------|--------------------------:|
| LogisticRegression |   0.74828 |     0.16002 |  0.67609 |    0.25878 | 136595 | 61370     |                    4.2895 |
|                    |           |             |      

# Avec feature engineering V2

## feature engineering

In [235]:
def feature_engineering(train, test):
    """Créer de nouvelles fonctionnalités à partir des données existantes."""
    train['AMT_ANNUITY/INCOME'] = train['AMT_ANNUITY'] / train['AMT_INCOME_TOTAL']
    test['AMT_ANNUITY/INCOME'] = test['AMT_ANNUITY'] / test['AMT_INCOME_TOTAL']
    
    train['credit_annuity_ratio'] = train['AMT_CREDIT'] / train['AMT_ANNUITY']
    test['credit_annuity_ratio'] = test['AMT_CREDIT'] / test['AMT_ANNUITY']
    
    train['credit_goods_price_ratio'] = train['AMT_CREDIT'] / train['AMT_GOODS_PRICE']
    test['credit_goods_price_ratio'] = test['AMT_CREDIT'] / test['AMT_GOODS_PRICE']
    
    train['credit_downpayment'] = train['AMT_GOODS_PRICE'] - train['AMT_CREDIT']
    test['credit_downpayment'] = test['AMT_GOODS_PRICE'] - test['AMT_CREDIT']
    
    train['AGE_INT'] = (train['DAYS_BIRTH'] / -365).astype(int)
    test['AGE_INT'] = (test['DAYS_BIRTH'] / -365).astype(int)
    
    return train, test


## Nettoyage

In [237]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer

def clean_data(train_file, test_file):
    """Nettoyer et préparer les données pour l'entraînement et le test."""
    print("Début du nettoyage des données...")
    
    app_train = pd.read_csv(train_file)
    app_test = pd.read_csv(test_file)
    
    label_encoders = {}
    for col in app_train.columns:
        if app_train[col].dtype == 'object':
            le = LabelEncoder()
            app_train[col] = le.fit_transform(app_train[col].astype(str))
            if col in app_test.columns:
                app_test[col] = le.transform(app_test[col].astype(str))
            label_encoders[col] = le
    
    app_train, app_test = feature_engineering(app_train, app_test)
    
    if 'TARGET' in app_train:
        train = app_train.drop(columns=['TARGET'])
        train_labels = app_train['TARGET']
    else:
        raise ValueError("La colonne 'TARGET' n'existe pas dans les données d'entraînement.")
    
    test = app_test.drop(columns=['SK_ID_CURR'])
    
    common_cols = train.columns.intersection(test.columns)
    train = train[common_cols]
    test = test[common_cols]
    
    imputer = SimpleImputer(strategy='median')
    train_numeric = imputer.fit_transform(train)
    test_numeric = imputer.transform(test)
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_numeric = scaler.fit_transform(train_numeric)
    test_numeric = scaler.transform(test_numeric)
    
    train_final = pd.DataFrame(train_numeric, columns=common_cols)
    test_final = pd.DataFrame(test_numeric, columns=common_cols)
    
    print("Fin du nettoyage des données.")
    return train_final, test_final, train_labels, app_test['TARGET']


## RFE

In [239]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

def apply_rfe_to_feature_engineering(X, y, engineered_features):
    """Appliquer RFE uniquement sur les caractéristiques issues du feature engineering."""
    X_engineered = X[engineered_features]
    
    clf = LogisticRegression(max_iter=5000)
    cv = StratifiedKFold(5)
    
    rfecv = RFECV(
        estimator=clf,
        step=1,
        cv=cv,
        scoring="accuracy",
        min_features_to_select=2,
        n_jobs=2
    )
    
    rfecv.fit(X_engineered, y)
    
    selected_features = [feature for feature, selected in zip(engineered_features, rfecv.support_) if selected]
    print(f"Optimal number of features: {rfecv.n_features_}")
    print(f"Selected features: {selected_features}")
    
    return selected_features


## Entraînement des Modèles

In [241]:
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

def train_models(train_final, train_labels, best_params):
    """Entraîner les modèles en utilisant les paramètres optimaux et RFE pour le feature engineering."""
    
    # Les caractéristiques issues du feature engineering
    engineered_features = [
        'AMT_ANNUITY/INCOME', 
        'credit_annuity_ratio', 
        'credit_goods_price_ratio', 
        'credit_downpayment', 
        'AGE_INT'
    ]
    
    # Appliquer RFE uniquement sur ces nouvelles caractéristiques
    selected_engineered_features = apply_rfe_to_feature_engineering(train_final, train_labels, engineered_features)
    
    # Conserver les autres caractéristiques non issues du feature engineering
    other_features = [col for col in train_final.columns if col not in engineered_features]
    
    # Créer une liste des caractéristiques finales à utiliser
    final_features = other_features + selected_engineered_features
    
    # Sélectionner les données d'entraînement finales
    X_train = train_final[final_features]
    
    # Entraîner les modèles avec les caractéristiques sélectionnées
    models = {}
    
    # Logistic Regression avec class_weight
    clf = LogisticRegression(**best_params['LogisticRegression'])
    clf.fit(X_train, train_labels)
    models['Logistic Regression'] = clf
    
    # CatBoost Classifier (pas de class_weight spécifique, mais CatBoost gère le déséquilibre en interne)
    clf_catboost = CatBoostClassifier(**best_params['CatBoostClassifier'])
    clf_catboost.fit(X_train, train_labels, verbose=0)
    models['CatBoost Classifier'] = clf_catboost
    
    return models, final_features


## Recherche des Hyperparamètres

In [243]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

def search_hyperparameters(train_final, train_labels):
    """Rechercher les meilleurs hyperparamètres pour les modèles."""
    print("Début de la recherche des hyperparamètres...")
    
    models = {}
    
    # Hyperparamètres pour Logistic Regression
    log_reg_params = {
        'C': [100, 10],
        'solver': ['newton-cg', 'lbfgs'],
        'penalty': ['l2'],
        'class_weight': ['balanced']  # Ajout de la pondération des classes
    }
    log_reg = LogisticRegression(max_iter=7000, class_weight='balanced')
    log_reg_grid = GridSearchCV(log_reg, log_reg_params, cv=5, scoring='roc_auc')
    log_reg_grid.fit(train_final, train_labels)
    models['LogisticRegression'] = log_reg_grid.best_params_
    
    # Hyperparamètres pour CatBoostClassifier
    catboost_params = {
        'depth': [1, 2, 3],
        'learning_rate': [0.01, 0.1],
        'l2_leaf_reg': [6,7],
        'bagging_temperature': [0.3, 0.2, 0.1],
        'loss_function': ['Logloss'],
        'auto_class_weights' : ['Balanced'],
    }
    catboost_model = CatBoostClassifier(iterations=10, verbose=0)
    catboost_grid = GridSearchCV(catboost_model, catboost_params, cv=5, scoring='roc_auc')
    catboost_grid.fit(train_final, train_labels)
    models['CatBoostClassifier'] = catboost_grid.best_params_
    
    print("Fin de la recherche des hyperparamètres.")
    return models


## Évaluation des Modèles

In [245]:
import time
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix

def measure_prediction_time(model, X, n_estimations=1000):
    """Mesurer le temps de prédiction pour un nombre donné d'estimations."""
    start_time = time.time()
    for _ in range(n_estimations):
        model.predict(X)
    end_time = time.time()
    total_time = end_time - start_time
    return total_time

def evaluate_models(models, X_train, y_train, X_test, y_test, selected_features):
    """Évaluer les modèles en termes de performances et de temps de prédiction, et afficher les meilleurs hyperparamètres."""
    
    metrics_train = []
    metrics_test = []

    for name, model in models.items():
        # Prédictions pour l'entraînement et le test
        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)
        
        # Calculer les probabilités pour ROC AUC
        train_proba = model.predict_proba(X_train)[:, 1]
        test_proba = model.predict_proba(X_test)[:, 1]

        # Mesurer le temps de prédiction pour 1000 estimations
        estimation_time_train = measure_prediction_time(model, X_train, n_estimations=1000)
        estimation_time_test = measure_prediction_time(model, X_test, n_estimations=1000)

        # Enregistrer les métriques pour les données d'entraînement
        metrics_train.append({
            'Model': name,
            'ROC AUC': round(roc_auc_score(y_train, train_proba), 5),
            'Precision': round(precision_score(y_train, train_pred), 5),
            'Recall': round(recall_score(y_train, train_pred), 5),
            'F1-Score': round(f1_score(y_train, train_pred), 5),
            'Confusion Matrix': str(confusion_matrix(y_train, train_pred)),
            'Time (1000 estimations)': round(estimation_time_train, 2)
        })

        # Enregistrer les métriques pour les données de test
        metrics_test.append({
            'Model': name,
            'ROC AUC': round(roc_auc_score(y_test, test_proba), 5),
            'Precision': round(precision_score(y_test, test_pred), 5),
            'Recall': round(recall_score(y_test, test_pred), 5),
            'F1-Score': round(f1_score(y_test, test_pred), 5),
            'Confusion Matrix': str(confusion_matrix(y_test, test_pred)),
            'Time (1000 estimations)': round(estimation_time_test, 2)
        })

        # Afficher les meilleurs hyperparamètres du modèle
        print(f"Best Hyperparameters for {name}:")
        print(model.get_params())

    return metrics_train, metrics_test


## Afichage resultats

In [247]:
from tabulate import tabulate

def display_results(metrics_train, metrics_test):
    """Afficher les résultats des modèles avec tabulate, en incluant l'estimation des 1000 prédictions et en formatant les scores."""

    # Affichage des résultats pour les données d'entraînement
    if metrics_train and isinstance(metrics_train, list) and all(isinstance(d, dict) for d in metrics_train):
        headers_train = metrics_train[0].keys()
        results_table_train = tabulate(
            [[f"{value:.5f}" if isinstance(value, (float, int)) else value for value in d.values()] for d in metrics_train],
            headers=headers_train,
            tablefmt='pipe',
            floatfmt=".5f"
        )
        print("\nMétriques pour les données d'entraînement :")
        print(results_table_train)
    else:
        print("Erreur : Les données pour l'entraînement ne sont pas au format attendu ou sont vides.")

    # Affichage des résultats pour les données de test
    if metrics_test and isinstance(metrics_test, list) and all(isinstance(d, dict) for d in metrics_test):
        headers_test = metrics_test[0].keys()
        results_table_test = tabulate(
            [[f"{value:.5f}" if isinstance(value, (float, int)) else value for value in d.values()] for d in metrics_test],
            headers=headers_test,
            tablefmt='pipe',
            floatfmt=".5f"
        )
        print("\nMétriques pour les données de test :")
        print(results_table_test)
    else:
        print("Erreur : Les données pour le test ne sont pas au format attendu ou sont vides.")


## Fonction Principale

In [249]:
def process_and_predict(train_file, test_file):
    """Fonction principale pour nettoyer les données, rechercher les hyperparamètres,
    entraîner les modèles, et évaluer les performances."""
    print("Début du traitement et des prédictions...")
    
    # Nettoyage des données
    train_final, test_final, train_labels, test_labels = clean_data(train_file, test_file)
    
    # Recherche des hyperparamètres
    best_params = search_hyperparameters(train_final, train_labels)
    
    # Entraînement des modèles
    models, selected_features = train_models(train_final, train_labels, best_params)
    
    # Évaluation des performances
    metrics_train, metrics_test = evaluate_models(models, train_final, train_labels, test_final, test_labels, selected_features)
    
    # Affichage des résultats
    display_results(metrics_train, metrics_test)
    
    print("Fin du traitement et des prédictions.")
    return metrics_train, metrics_test


## Appel

In [251]:
# Appel de la fonction principale avec les chemins des fichiers
metrics_train, metrics_test = process_and_predict('data/application_train_split.csv', 'data/application_test_split.csv')


Début du traitement et des prédictions...
Début du nettoyage des données...
Fin du nettoyage des données.
Début de la recherche des hyperparamètres...


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


Fin de la recherche des hyperparamètres.
Optimal number of features: 5
Selected features: ['AMT_ANNUITY/INCOME', 'credit_annuity_ratio', 'credit_goods_price_ratio', 'credit_downpayment', 'AGE_INT']
Best Hyperparameters for Logistic Regression:
{'C': 100, 'class_weight': 'balanced', 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Best Hyperparameters for CatBoost Classifier:
{'learning_rate': 0.1, 'depth': 3, 'l2_leaf_reg': 7, 'loss_function': 'Logloss', 'auto_class_weights': 'Balanced', 'bagging_temperature': 0.3}

Métriques pour les données d'entraînement :
| Model               |   ROC AUC |   Precision |   Recall |   F1-Score | Confusion Matrix   |   Time (1000 estimations) |
|:--------------------|----------:|------------:|---------:|-----------:|:-------------------|------------------